In [1]:
import sqlite3 as sq
import pandas as pd
import numpy as np
cnx = sq.connect('test.db')
cur = cnx.cursor()

In [ ]:
masterpatientlist = pd.read_csv('MRN_Patient_List.txt',
                                sep='\,',lineterminator='\n',
                                usecols=[0,1,2,3,4,5,6],
                                converters={'current_mrn':str, 'new_mrn':str, 'legacy_mrn':str},
                                engine = 'python'
)


In [ ]:
masterpatientlist.info()

In [ ]:
masterpatientlist.head()

##  PACS1 STUFF BELOW

In [ ]:
patientextract = pd.read_excel('patientextract04152019.xlsx')
patientextract.info()

In [ ]:
patientextract.head()

In [ ]:
patient = pd.read_csv('mapping.txt',sep='\,',lineterminator='\n',encoding='utf-16')

In [ ]:
patient.head()

In [ ]:
patient.info()

In [ ]:
patient.drop_duplicates(subset='StudyInstanceUID', inplace=True)
patient.drop_duplicates(subset='MedicalRecordNumber', inplace=True)

In [ ]:
patient.info()

In [ ]:
patientmaster = patient.copy()

In [ ]:
patientmaster['mapping'] = 'NEEDS ATTENTION'
patientmaster['status'] = 'NEEDS ATTENTION'

In [ ]:
testpatient = [
'list'
]

In [ ]:
testpatientmrn = patientmaster['MedicalRecordNumber'][patientmaster['StudyInstanceUID'].isin(testpatient)]

In [ ]:
testpatientsuidcomplete = patient['StudyInstanceUID'][patient['MedicalRecordNumber'].isin(testpatientmrn)]

In [ ]:
testpatientsuidcomplete.shape

In [ ]:
testpatientsuidcomplete.to_csv("C:\\temp\\test.csv")

In [ ]:
patientmaster.loc[patientmaster['StudyInstanceUID'].isin(testpatient) , ['mapping','status']] = 'DELETE_FROM_LIST', 'DELETE_FROM_LIST'

In [ ]:
patientmaster.status.value_counts()

In [ ]:
patient1 = patient[~patient['StudyInstanceUID'].isin(testpatient)].copy()
patient1.shape

In [ ]:
patient1['MappedMedicalRecordNumber'] = patient1['MedicalRecordNumber']
patient1['MappedMedicalRecordNumber'].replace('-','',inplace=True,regex=True)
patient1['MappedMedicalRecordNumber'] = patient1['MappedMedicalRecordNumber'].map('{:0>12}'.format) 
patient1.shape

In [ ]:
patient1.head()

In [ ]:
a = patient1.merge(masterpatientlist, how='inner', left_on = 'MappedMedicalRecordNumber', right_on = 'current_mrn')
a.shape

In [ ]:
adict = a.set_index('MedicalRecordNumber').to_dict()['current_mrn']

In [ ]:
a[a['patient_name'].isnull()]

In [ ]:
patientmaster['mapping'][patientmaster['StudyInstanceUID'].isin(a['StudyInstanceUID'])] = patientmaster['MedicalRecordNumber'].map(adict)
patientmaster['status'][patientmaster['StudyInstanceUID'].isin(a['StudyInstanceUID'])] = 'CURRENT_MRN_MATCH'

In [ ]:
patient2 = patient1[~patient1['StudyInstanceUID'].isin(a['StudyInstanceUID'])]
patient2.shape

In [ ]:
b = patient2.merge(masterpatientlist, how='inner', left_on = 'MappedMedicalRecordNumber', right_on = 'legacy_mrn')
b.shape

In [ ]:
neednamechange = b[b['patient_name'].isnull()]

In [ ]:
bdict = b.set_index('MedicalRecordNumber').to_dict()['current_mrn']

In [ ]:
patientmaster['mapping'][patientmaster['StudyInstanceUID'].isin(b['StudyInstanceUID'])] = patientmaster['MedicalRecordNumber'].map(bdict)
patientmaster['status'][patientmaster['StudyInstanceUID'].isin(b['StudyInstanceUID'])] = 'LEGACY_MRN_MATCH'

In [ ]:
patient3 = patient2[~patient2['StudyInstanceUID'].isin(b['StudyInstanceUID'])]
patient3.shape

In [ ]:
c = patient3.merge(patientextract, how='inner', left_on='MedicalRecordNumber',right_on='PACS1_Original_MRN')
c.info()

In [ ]:
masterpatientlist.info()

In [ ]:
masterpatientlistclean = masterpatientlist.dropna(subset=['last_name','first_name','birth_date','gender']).copy()
masterpatientlistclean.info()

In [ ]:
masterpatientlistclean.head()

In [ ]:
masterpatientlistclean['birth_date'] = pd.to_datetime(masterpatientlistclean['birth_date'], errors='coerce')
masterpatientlistclean = masterpatientlistclean.dropna(subset=['last_name','first_name','birth_date','gender'])
masterpatientlistclean['birth_date'] = masterpatientlistclean.birth_date.apply(lambda x: x.strftime('%Y%m%d')).astype(int)
masterpatientlistclean['first_name'] = masterpatientlistclean['first_name'].str.upper()
masterpatientlistclean['last_name'] = masterpatientlistclean['last_name'].str.upper()
masterpatientlistclean.info()

In [ ]:
masterpatientlistclean.head()

In [ ]:
c.columns

In [ ]:
cclean = c.dropna(subset=['LastName','FirstName','BirthDate','Gender']).copy()
cclean['FirstName'] = cclean['FirstName'].str.upper()
cclean['LastName'] = cclean['LastName'].str.upper()
cclean.info()

In [ ]:
cclean['BirthDate'] = cclean['BirthDate'].astype(int)
cclean.info()

In [ ]:
d = cclean.merge(masterpatientlistclean, how='inner', left_on = ['LastName','FirstName','BirthDate','Gender'], right_on = ['last_name','first_name','birth_date','gender'])
d.info()

In [ ]:
d[['LastName','FirstName','BirthDate','Gender','last_name','first_name','birth_date','gender','current_mrn']]

In [ ]:
e = cclean.merge(masterpatientlistclean, how='inner', left_on = ['LastName','FirstName','BirthDate'], right_on = ['last_name','first_name','birth_date'])
e.info()

In [ ]:
e[['LastName','FirstName','BirthDate','Gender','last_name','first_name','birth_date','gender','current_mrn']]

In [ ]:
lastbirthgender = cclean.merge(masterpatientlistclean, how='inner', left_on = ['LastName','BirthDate','Gender'], right_on = ['last_name','birth_date','gender'])
lastbirthgender.info()

In [ ]:
lastbirthgender[['patient_name','MedicalRecordNumber','MappedMedicalRecordNumber','LastName','FirstName','BirthDate','Gender','first_name','last_name','birth_date','gender','current_mrn']].to_csv("C:\\Users\\username\\desktop\\lastbirthgender.csv", index=False)

In [ ]:
lastbirthgendermanual = pd.read_excel('lastbirthgendermanual.xlsx', converters = {'current_mrn':str})
lastbirthgendermanual.info()

In [ ]:
lastbirthgendermanual.head()

In [ ]:
lastbirthgendermanualdict =  lastbirthgendermanual.set_index('MedicalRecordNumber').to_dict()['current_mrn']

In [ ]:
patientmaster['mapping'][patientmaster['MedicalRecordNumber'].isin(lastbirthgendermanual['MedicalRecordNumber'])] = patientmaster['MedicalRecordNumber'].map(lastbirthgendermanualdict)
patientmaster['status'][patientmaster['MedicalRecordNumber'].isin(lastbirthgendermanual['MedicalRecordNumber'])] = 'LAST_BIRTHDATE_GENDER_MATCH'

In [ ]:
firstbirthgender = cclean.merge(masterpatientlistclean, how='inner', left_on = ['FirstName','BirthDate','Gender'], right_on = ['first_name','birth_date','gender'])
firstbirthgender = firstbirthgender[~firstbirthgender['StudyInstanceUID'].isin(lastbirthgender['StudyInstanceUID'])]
firstbirthgender.info()

In [ ]:
firstbirthgender[['patient_name','MedicalRecordNumber','MappedMedicalRecordNumber','LastName','FirstName','BirthDate','Gender','first_name','last_name','birth_date','gender','current_mrn']].to_csv("C:\\Users\\username\\desktop\\firstbirthgender.csv", index=False)

In [ ]:
firstbirthgendermanual = pd.read_excel('firstbirthgendermanual.xlsx', converters = {'current_mrn':str})
firstbirthgendermanual.info()

In [ ]:
firstbirthgendermanualdict =  firstbirthgendermanual.set_index('MedicalRecordNumber').to_dict()['current_mrn']
firstbirthgendermanualstatusdict = firstbirthgendermanual.set_index('MedicalRecordNumber').to_dict()['STATUS']

In [ ]:
patientmaster['mapping'][patientmaster['MedicalRecordNumber'].isin(firstbirthgendermanual['MedicalRecordNumber'])] = patientmaster['MedicalRecordNumber'].map(firstbirthgendermanualdict)
patientmaster['status'][patientmaster['MedicalRecordNumber'].isin(firstbirthgendermanual['MedicalRecordNumber'])] = patientmaster['MedicalRecordNumber'].map(firstbirthgendermanualstatusdict)

In [ ]:
remaining = patientmaster[patientmaster['status']=='NEEDS ATTENTION']
remaining.to_csv("C:\\Users\\username\\desktop\\remaining.csv", index=False)

In [ ]:
patient4 = patient3[patient3['StudyInstanceUID'].isin(remaining['StudyInstanceUID'])]
patient4.shape

In [ ]:
patient4['MappedMedicalRecordNumber'] = patient4['MappedMedicalRecordNumber'] + ''

In [ ]:
patient4dict = patient4.set_index('MedicalRecordNumber').to_dict()['MappedMedicalRecordNumber']

In [ ]:
patientmaster['mapping'][patientmaster['MedicalRecordNumber'].isin(patient4['MedicalRecordNumber'])] = patientmaster['MedicalRecordNumber'].map(patient4dict)
patientmaster['status'][patientmaster['MedicalRecordNumber'].isin(patient4['MedicalRecordNumber'])] = 'REVIEW_AFTER_PUSH'

In [ ]:
patientmaster.status.value_counts()

In [ ]:
patientmaster.to_csv("C:\\Users\\username\\desktop\\patientmaster.csv", index=False)

In [ ]:
neednamechange.to_csv("C:\\Users\\username\\desktop\\neednamechange.csv", index=False)

In [ ]:
updatedmapping = patientmaster[['patient_name','MedicalRecordNumber','mapping','status']][patientmaster['status'] != 'NEEDS ATTENTION']
updatedmapping.to_csv("C:\\Users\\username\\desktop\\updatedmapping.csv", index=False)
updatedmapping.shape